In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import lightgbm as lgb

import catboost
from catboost import CatBoostRegressor, CatBoostClassifier
from catboost import Pool

import sklearn
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, train_test_split, KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve

import datetime
from datetime import datetime, timedelta, timezone
import holidays
import matplotlib.pyplot as plt
from tqdm import tqdm

from statistics import mean
np.random.seed(42)

In [2]:
# Read cvs
job_details = pd.read_csv('../input/armutdac2022/JOBDETAILS.csv')
job_questions = pd.read_csv('../input/armutdac2022/JOBQUESTIONS.csv')
pro_loc = pd.read_csv('../input/armutdac2022/PROLOC.csv')
test = pd.read_csv('../input/armutdac2022/TEST.csv')
train = pd.read_csv('../input/armutdac2022/TRAIN.csv')
sample = pd.read_csv('../input/armutdac2022/sample sub.csv')

# Concatenate all data for easier feature engineering
all_data = pd.concat((train, test), axis=0)
all_data = all_data.merge(job_details.rename(columns={'OUTER_LOC': 'JOB_OUTER_LOC', 'INNER_LOC': 'JOB_INNER_LOC'}), on='JOBID', how='left')
all_data = all_data.merge(pro_loc.rename(columns={'OUTER_LOC': 'PRO_OUTER_LOC', 'INNER_LOC': 'PRO_INNER_LOC'}), on='PROID', how='left')

In [3]:
# Remove time zones as they are all the same & sort the data by OPP_CREATEDATE column
def remove_time_zone(x):
    return x[:-6]

all_data['OPP_CREATEDATE'] = all_data['OPP_CREATEDATE'].apply(remove_time_zone)
all_data['JOB_CREATEDATE'] = all_data['JOB_CREATEDATE'].apply(remove_time_zone)

all_data['OPP_CREATEDATE'] = pd.to_datetime(all_data['OPP_CREATEDATE'])
all_data['JOB_CREATEDATE'] = pd.to_datetime(all_data['JOB_CREATEDATE'])

all_data.sort_values('OPP_CREATEDATE', inplace=True)

In [4]:
# Create columns for each question in job_questions
dummy = pd.get_dummies(job_questions['QUESTIONID']).multiply(job_questions['ANSWERID'], axis="index")
questions_dict = {}
for key in dummy.columns:
    questions_dict[key] = 'Q_' + str(key)
dummy.rename(columns=questions_dict, inplace=True)
job_questions = pd.concat((job_questions, dummy), axis=1).drop(columns=['QUESTIONID', 'ANSWERID']).groupby('JOBID').max().reset_index()

In [5]:
all_data = all_data.merge(job_questions, on='JOBID', how='left')

In [6]:
# Get time based features from OPP_CREATEDATE & JOB_CREATEDATE
all_data['OPP_MONTH'] = all_data['OPP_CREATEDATE'].dt.month
all_data['JOB_MONTH'] = all_data['JOB_CREATEDATE'].dt.month

all_data['OPP_DAY'] = all_data['OPP_CREATEDATE'].dt.day
all_data['JOB_DAY'] = all_data['JOB_CREATEDATE'].dt.day

all_data['OPP_WEEKDAY'] = all_data['OPP_CREATEDATE'].dt.weekday
all_data['JOB_WEEKDAY'] = all_data['JOB_CREATEDATE'].dt.weekday

all_data['OPP_HOUR'] = all_data['OPP_CREATEDATE'].dt.hour
all_data['JOB_HOUR'] = all_data['JOB_CREATEDATE'].dt.hour

all_data['TIME_DIFF'] = (all_data['OPP_CREATEDATE'] - all_data['JOB_CREATEDATE']).dt.total_seconds()

In [7]:
all_data['BOTH_OUTER_LOC'] = all_data['JOB_OUTER_LOC'] + '-' +  all_data['PRO_OUTER_LOC']

In [8]:
all_data['IS_SAME_HOUR'] = (all_data['JOB_HOUR'] == all_data['OPP_HOUR']).astype(int)
all_data['IS_SAME_DAY'] = (all_data['JOB_DAY'] == all_data['OPP_DAY']).astype(int)
all_data['IS_SAME_OUTER_LOC'] = (all_data['JOB_OUTER_LOC'] == all_data['PRO_OUTER_LOC']).astype(int)
all_data['IS_SAME_INNER_LOC'] = (all_data['JOB_INNER_LOC'] == all_data['PRO_INNER_LOC']).astype(int)

In [9]:
# Is it holiday?
holiday_list = list(holidays.Turkey(years=2021).keys())
all_data['OPP_IS_HOLIDAY'] = (all_data['OPP_CREATEDATE'].apply(lambda x: x in holiday_list)).astype(int)
all_data['JOB_IS_HOLIDAY'] = (all_data['JOB_CREATEDATE'].apply(lambda x: x in holiday_list)).astype(int)

# Is it weekend?
all_data['OPP_IS_WEEKEND'] = (all_data['OPP_WEEKDAY'] > 4).astype(int)
all_data['JOB_IS_WEEKEND'] = (all_data['JOB_WEEKDAY'] > 4).astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  after removing the cwd from sys.path.


In [10]:
def get_num(x):
    return x[5:]

def get_city(x):
    return x[5:-2]

all_data['JOB_OUTER_LOC_NUM'] = all_data['JOB_OUTER_LOC'].apply(get_num).astype(int)
all_data['JOB_INNER_LOC_NUM'] = all_data['JOB_INNER_LOC'].apply(get_num).astype(int)
all_data['PRO_OUTER_LOC_NUM'] = all_data['PRO_OUTER_LOC'].fillna('loc1_-999').apply(get_num).astype(int)
all_data['PRO_INNER_LOC_NUM'] = all_data['PRO_INNER_LOC'].fillna('loc2_-999').apply(get_num).astype(int)

all_data['OUTER_LOC_NUM_ABS_DIFF'] = np.abs(all_data['JOB_OUTER_LOC_NUM'] - all_data['PRO_OUTER_LOC_NUM'])
all_data['OUTER_LOC_NUM_DIFF'] = all_data['JOB_OUTER_LOC_NUM'] - all_data['PRO_OUTER_LOC_NUM']

all_data['INNER_LOC_NUM_ABS_DIFF'] = np.abs(all_data['JOB_INNER_LOC_NUM'] - all_data['PRO_INNER_LOC_NUM'])
all_data['INNER_LOC_NUM_DIFF'] = all_data['JOB_INNER_LOC_NUM'] - all_data['PRO_INNER_LOC_NUM']

all_data['JOB_OUTER_LOC_CT'] = all_data['JOB_OUTER_LOC'].apply(get_city).astype(int)
all_data['PRO_OUTER_LOC_CT'] = all_data['PRO_OUTER_LOC'].fillna('loc1_-999').apply(get_city).astype(int)

all_data['JOB_OUTER_LOC_CT_IS_SAME'] = (all_data['PRO_OUTER_LOC_CT'] == all_data['JOB_OUTER_LOC_CT']).astype(int)

In [11]:
def get_target_based_features(all_data, cols=['JOBID', 'PROID', 'SERVICEID', 'JOB_OUTER_LOC', 'PRO_OUTER_LOC'
                                              ,'BOTH_OUTER_LOC'
                                             ]):
    for col in tqdm(cols):
        #print(col)
        all_data[col+'_CUMCOUNT'] = all_data.groupby(col).cumcount()
        
        if  col != 'JOBID':
            all_data[col+'_KNOWN_CUMCOUNT'] = all_data[col+'_CUMCOUNT']
            ind = all_data[all_data['ISQUOTED'].isna()].index[0]
            all_data.loc[ind:, col+'_KNOWN_CUMCOUNT'] = np.nan
            all_data.loc[ind:, col+'_KNOWN_CUMCOUNT'] = all_data.groupby(col).ffill().loc[ind:,col+'_KNOWN_CUMCOUNT'] + 1
    
            all_data[col+'_QUOTED_CUMSUM'] = all_data.fillna(0).groupby([col]).cumsum()['ISQUOTED'] - all_data['ISQUOTED'].fillna(0)
            all_data[col+'_NOTQUOTED_CUMSUM'] = all_data[col+'_CUMCOUNT'] - all_data[col+'_QUOTED_CUMSUM']
            all_data[col+'_QUOTED_PER'] = all_data[col+'_QUOTED_CUMSUM'] / all_data[col+'_KNOWN_CUMCOUNT']
            
            all_data[col+'_LAST_ISQUOTED'] = all_data.groupby(col).shift()['ISQUOTED']
            all_data[col+'_LAST_ISQUOTED'] = all_data.groupby(col)[col+'_LAST_ISQUOTED'].ffill()
            
        all_data[col+'_TIME_SINCE_LAST_OPP'] = all_data.groupby(col)['OPP_CREATEDATE'].diff().dt.total_seconds()
        all_data[col+'_TIME_SINCE_FIRST_OPP'] = (all_data['OPP_CREATEDATE'] - all_data.groupby(col)['OPP_CREATEDATE'].transform('first')).dt.total_seconds()
        all_data[col+'_TIME_SINCE_LAST_JOB'] = all_data.groupby(col)['JOB_CREATEDATE'].diff().dt.total_seconds()
        all_data[col+'_TIME_SINCE_FIRST_JOB'] = (all_data['JOB_CREATEDATE'] - all_data.groupby(col)['JOB_CREATEDATE'].transform('first')).dt.total_seconds()
    
    return all_data

In [12]:
def get_last_job(all_data):
    
    all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = np.nan
    all_data.loc[all_data[all_data['ISQUOTED'] == 1].index, 'LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = all_data[all_data['ISQUOTED'] == 1]['JOB_OUTER_LOC_NUM']
    all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = all_data.groupby('PROID').ffill()['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID']
    all_data.loc[all_data[all_data['ISQUOTED'] == 1].index, 'LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = np.nan
    all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = all_data.groupby('PROID').ffill()['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID']
    #all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'].fillna(-99)
    #le = LabelEncoder()
    #all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'] = le.fit_transform(all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID'].astype(str))

    return all_data

In [13]:
cat_cols = ['PROID', 'SERVICEID', 'JOB_OUTER_LOC', 'JOB_INNER_LOC', 'PRO_OUTER_LOC',
            'PRO_INNER_LOC', 'BOTH_OUTER_LOC'
            #, 'PROID-JOB_OUTER_LOC'
           ]
for col in all_data.columns:
    if col[:2] == 'Q_': 
        cat_cols.append(col)
to_drop_cols = ['JOBID', 
                'OPP_CREATEDATE', 
                'JOB_CREATEDATE', 'ISQUOTED']

for col in cat_cols:
    #print(col)
    if all_data[col].dtype == int:
        all_data[col].fillna(-99, inplace=True)
    elif all_data[col].dtype == float:
        all_data[col] = all_data[col].fillna(-99).astype(int)
    else:
        all_data[col].fillna('nan', inplace=True)

In [14]:
def get_best_threshold(y, preds):
    best_score = 0.0
    best_thres = 0.0
    for i in range(10, 30, 1):
        thres = i/100
        score = f1_score(y, (preds > thres).astype(int))
        #print(best_score)
        if score > best_score:
            best_thres = thres
            best_score = score
    print(best_thres)
    print(best_score)
    return best_thres

In [15]:
all_data = get_target_based_features(all_data)
all_data = get_last_job(all_data)
#datetimes = [datetime(2021, 9, 16), datetime(2021, 9, 23), datetime(2021, 9, 30), datetime(2021, 10, 6)]

100%|██████████| 6/6 [02:22<00:00, 23.68s/it]


In [16]:
all_data['LAST_JOB_OUTER_LOC_QUOTED_BY_PROID']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
733226       NaN
733227       NaN
733228       NaN
733229    2193.0
733230     558.0
Name: LAST_JOB_OUTER_LOC_QUOTED_BY_PROID, Length: 733231, dtype: float64

In [17]:
#all_weights = pd.DataFrame(all_data.index)
#all_weights[0] = np.log10(all_weights[0] + 1000)

In [18]:
## Folds start here
all_train = all_data[all_data['ISQUOTED'].notna()]
test = all_data[all_data['ISQUOTED'].isna()]
test_preds = pd.DataFrame(index=test.index)
test_preds['JOBID_PROID'] = test['JOBID'].astype(str) + '-' + test['PROID'].astype(str)
test_preds['ISQUOTED'] = 0.0
best_thres = 0.0
skf = StratifiedKFold(5)
for train_index, val_index in skf.split(all_train, all_train['ISQUOTED']):
    
    train = all_data.iloc[train_index]
    #weights = all_weights.iloc[train_index]
    val = all_data.iloc[val_index]
    
    train_pool = Pool(data=train.drop(columns=to_drop_cols), 
                      #weight=weights[0], 
                      label=train['ISQUOTED'], cat_features=cat_cols)
    val_pool = Pool(data=val.drop(columns=to_drop_cols), label=val['ISQUOTED'], cat_features=cat_cols)
    model = CatBoostClassifier(iterations=1000, 
                              learning_rate=0.07,
                              depth=5,
                              subsample=0.8,
                              min_child_samples=0,
                              loss_function='CrossEntropy',
                              eval_metric='AUC',
                              use_best_model=True, 
                              has_time=True,
                              random_state=42)
    model.fit(train_pool,
              eval_set=val_pool, 
              verbose=50,
              plot=False)
    val_to_pred = val.drop(columns=to_drop_cols)
    preds = model.predict_proba(val_to_pred)
    best_thres = get_best_threshold(val['ISQUOTED'], preds[:,1])
    #for_test = get_last_job(for_test)
    test_preds['ISQUOTED'] += (model.predict_proba(test.drop(columns=to_drop_cols))[:,1] > get_best_threshold(val['ISQUOTED'], preds[:,1])).astype(int)

0:	test: 0.7104636	best: 0.7104636 (0)	total: 1.04s	remaining: 17m 21s
50:	test: 0.8683246	best: 0.8683246 (50)	total: 42.9s	remaining: 13m 17s
100:	test: 0.8810984	best: 0.8810984 (100)	total: 1m 26s	remaining: 12m 51s
150:	test: 0.8843507	best: 0.8843507 (150)	total: 2m 9s	remaining: 12m 10s
200:	test: 0.8859807	best: 0.8859807 (200)	total: 2m 53s	remaining: 11m 28s
250:	test: 0.8872716	best: 0.8872716 (250)	total: 3m 36s	remaining: 10m 45s
300:	test: 0.8885150	best: 0.8885150 (300)	total: 4m 20s	remaining: 10m 4s
350:	test: 0.8889640	best: 0.8889904 (347)	total: 5m 3s	remaining: 9m 21s
400:	test: 0.8893386	best: 0.8893386 (400)	total: 5m 45s	remaining: 8m 36s
450:	test: 0.8894561	best: 0.8894883 (442)	total: 6m 27s	remaining: 7m 51s
500:	test: 0.8895887	best: 0.8896903 (473)	total: 7m 11s	remaining: 7m 9s
550:	test: 0.8896598	best: 0.8898069 (508)	total: 7m 54s	remaining: 6m 26s
600:	test: 0.8880181	best: 0.8900012 (570)	total: 8m 36s	remaining: 5m 42s
650:	test: 0.8880634	best: 0.8

In [19]:
test_preds['ISQUOTED'] = (test_preds['ISQUOTED'] > 2).astype(int)
#best_thres /= 5
#test_preds['ISQUOTED'] = (test_preds['ISQUOTED'] > best_thres).astype(int)
test_preds.to_csv('adv_v87.csv', index=False)